In [1]:
import pandas as pd
from ds_util.qubole import run_presto

In [2]:
df=pd.read_csv('ibotta_full_walmart_new_list_2020-09-21.txt')

In [3]:
df.head()

,00000091C29D91C93B200BA5A6C7DC37CC6023781569FF2F816710BD793333DA
0,0000029907C8BDD1B920E3709B4CE891B8CEF1D324BFBC...
1,000003541DA609333E4FB096CA0F49CF1E43FD2BA1DC67...
2,0000038A3AFB972B873C0B21730FC7FB3FF7D3F47A394F...
3,000004EC8B22916851013178ADB500256C69D5372EBE50...
4,000005FEFBDBD194EC4F9EEC192A679FF47EDD740FA1CF...


In [ ]:
df.count()

In [ ]:
#link to the normalization function in Scala
https://api.qubole.com/notebooks#home?id=185446&type=my-notebooks&view=home

In [4]:
hashed_emails=run_presto("""select distinct customer_id, to_hex(sha256(to_utf8(normalized))) as hashed_email from retail.wm_normalized_20210409""", return_data=True)

In [5]:
hashed_emails

,customer_id,hashed_email
0,1559274,EA8E5FBBDFDA9724B7BAD94BD82D749AFF6CB45E8E4166...
1,25917283,CC8025DA35A2404ED4C94B531BC4AA572F02044B514134...
2,25919312,A37DBC7FD3D88B6E3ED27825FC2EF67C7075D7BA1AC386...
3,27292191,F55726607AB442A3FA1A91FC3FCB619E82BBDD1590038B...
4,2501460,B5BCB1F74251A03F9589459647C2F634AAC68629C2233D...
...,...,...
40181617,24878631,6643743FD6FBCC6DF0AF2F2A42478F812878A065DBCBEC...
40181618,24880615,9AA9264736AAEF703A29CD07A66972E02A9DFCBD76D0ED...
40181619,2689770,F547021DBA11AF4403EAC430D42292AFA440221CFFCA87...
40181620,2694125,CB1820F4C8724462AAA089C34CB710BBE0287C5ACD524A...


In [6]:
new_to_wm=pd.merge(df, hashed_emails, left_on='00000091C29D91C93B200BA5A6C7DC37CC6023781569FF2F816710BD793333DA', right_on='hashed_email')

In [ ]:
14467582- 14474472

In [ ]:
#linked accounts query: https://api.qubole.com/v2/analyze?command_id=453792163

In [7]:
known_opd=run_presto("""select distinct cast(customer_id as bigint) customer_id from mhackler.wm_linked_accounts
union
select distinct customer_id from vw_receipts where retailer_id in (1408, 1678 )""", return_data=True)

In [9]:
known_opd.count()

customer_id    1035645
dtype: int64

In [10]:
new_to_wm.head()

,00000091C29D91C93B200BA5A6C7DC37CC6023781569FF2F816710BD793333DA,customer_id,hashed_email
0,0000029907C8BDD1B920E3709B4CE891B8CEF1D324BFBC...,4397919,0000029907C8BDD1B920E3709B4CE891B8CEF1D324BFBC...
1,000003541DA609333E4FB096CA0F49CF1E43FD2BA1DC67...,12738949,000003541DA609333E4FB096CA0F49CF1E43FD2BA1DC67...
2,0000038A3AFB972B873C0B21730FC7FB3FF7D3F47A394F...,35066202,0000038A3AFB972B873C0B21730FC7FB3FF7D3F47A394F...
3,000004EC8B22916851013178ADB500256C69D5372EBE50...,35046024,000004EC8B22916851013178ADB500256C69D5372EBE50...
4,000005FEFBDBD194EC4F9EEC192A679FF47EDD740FA1CF...,36338773,000005FEFBDBD194EC4F9EEC192A679FF47EDD740FA1CF...


In [8]:
segment=new_to_wm[~new_to_wm['customer_id'].isin(known_opd.customer_id)]

In [12]:
new_to_wm.count()

00000091C29D91C93B200BA5A6C7DC37CC6023781569FF2F816710BD793333DA    14474472
customer_id                                                         14474472
hashed_email                                                        14474472
dtype: int64

In [13]:
segment.count()

00000091C29D91C93B200BA5A6C7DC37CC6023781569FF2F816710BD793333DA    14321064
customer_id                                                         14321064
hashed_email                                                        14321064
dtype: int64

In [26]:
known_opd_launch=run_presto("""select distinct cast(customer_id as bigint) customer_id from mhackler.wm_linked_accounts
union
select distinct customer_id from vw_receipts where retailer_id in (1408, 1678 )
union
select distinct customer_id from vw_customer_retailer_launches where retailer_id in (1408, 1678)
""", return_data=True)

In [20]:
known_opd_launch.count()

customer_id    1174863
dtype: int64

In [27]:
segment2=new_to_wm[~new_to_wm['customer_id'].isin(known_opd_launch.customer_id)]

In [18]:
segment2.count()

00000091C29D91C93B200BA5A6C7DC37CC6023781569FF2F816710BD793333DA    14305494
customer_id                                                         14305494
hashed_email                                                        14305494
dtype: int64

In [19]:
segment.count()

00000091C29D91C93B200BA5A6C7DC37CC6023781569FF2F816710BD793333DA    14321064
customer_id                                                         14321064
hashed_email                                                        14321064
dtype: int64

In [12]:
new_after_match=run_presto("""select distinct id customer_id from vw_customers 
where created_at>date '2020-10-01'""", return_data=True)

In [28]:
segment2b=new_after_match[~new_after_match['customer_id'].isin(known_opd_launch.customer_id)]

In [29]:
total_a=pd.concat([segment2[['customer_id']],segment2b])

In [30]:
total_a.customer_id.nunique()

17406707

In [31]:
#checdk for overlap
pd.merge(total_a, known_opd_launch.customer_id, on='customer_id')

,customer_id


In [32]:
import numpy as np
total_a['segment']='new_to_wm_opd20210511'

In [33]:
sessioners=run_presto("""select distinct customer_id from vw_tracking_session where day>=current_date
-interval'90' day""", return_data=True)

In [34]:
total_a.head()

,customer_id,segment
0,4397919,new_to_wm_opd20210511
1,12738949,new_to_wm_opd20210511
2,35066202,new_to_wm_opd20210511
3,35046024,new_to_wm_opd20210511
4,36338773,new_to_wm_opd20210511


In [56]:
import numpy as np
split_list=np.array_split(total_a['customer_id'],2)
a=pd.DataFrame(split_list[0])
a['segment']='new_to_wm_opd20210511_p1'
a.to_csv('walmart_opd_new_bizops_p1_20210511.csv', index=False)
b=pd.DataFrame(split_list[1])
b['segment']='new_to_wm_opd20210511_p2'
b.to_csv('walmart_opd_new_bizops_p2_20210511.csv', index=False)


In [58]:
check=pd.read_csv('walmart_opd_new_bizops_p1_20210511.csv')
check

,customer_id,segment
0,4397919,new_to_wm_opd20210511_p1
1,12738949,new_to_wm_opd20210511_p1
2,35066202,new_to_wm_opd20210511_p1
3,35046024,new_to_wm_opd20210511_p1
4,36338773,new_to_wm_opd20210511_p1
...,...,...
8703786,319284,new_to_wm_opd20210511_p1
8703787,18609422,new_to_wm_opd20210511_p1
8703788,20793553,new_to_wm_opd20210511_p1
8703789,27710111,new_to_wm_opd20210511_p1


In [36]:
sessioners_wm=pd.merge(total_a, sessioners, on='customer_id', how='inner')
sessioners_wm.count()

customer_id    1711089
segment        1711089
dtype: int64

In [59]:
import numpy as np
sessioners_wm.rename(columns={'customer_id':'external_id'}, inplace=True)
split_list=np.array_split(sessioners_wm['external_id'],2)
pd.DataFrame(split_list[0]).to_csv('walmart_opd_new_marketing_p1_20210511.csv', index=False)
pd.DataFrame(split_list[1]).to_csv('walmart_opd_new_marketing_p2_20210511.csv', index=False)

In [60]:
pd.DataFrame(split_list[1])

,external_id
855545,39518471
855546,39518724
855547,39518845
855548,39519447
855549,39520071
...,...
1711084,37415460
1711085,37416686
1711086,37417046
1711087,37417206


In [53]:
b=pd.DataFrame(a)

,external_id
855545,39518471
855546,39518724
855547,39518845
855548,39519447
855549,39520071
...,...
1711084,37415460
1711085,37416686
1711086,37417046
1711087,37417206
